In [ ]:
import pandas as pd

# Charger le fichier Excel
df = pd.read_excel("path to Artifacts/Experiments and Results/Annotation_evaluation/manual_evaluation_cta/annotation.xlsx")  # remplace par ton vrai nom de fichier




In [ ]:
df

In [ ]:
def compute_metrics(column):
    total = df[column].notna().sum()
    exact = (df[column] == "match exact").sum()
    specific = (df[column] == "spécifique").sum()
    generic = (df[column] == "générique").sum()

    return {
        "total": total,
        "match_exact": exact,
        "spécifique": specific,
        "générique": generic,
        "prop_spécifique": specific / total if total else 0,
        "prop_générique": generic / total if total else 0,
        "précision": (exact + specific + generic) / total if total else 0,
    }

hybride_stats = compute_metrics("Remarque")       # colonne D
doduo_stats = compute_metrics("Remarque.1")       # colonne G

print("Hybride:", hybride_stats)
print("Doduo:", doduo_stats)

In [ ]:
import pandas as pd

def compute_metrics(df, col_colname, annotation, col_remarque, levels=["match exact", "spécifique", "générique"]):
    """
    Calcule précision, rappel, F1-score pour chaque niveau de granularité :
    - exact seulement
    - exact + spécifique
    - exact + spécifique + générique
    """

    results = []
    all_columns = df[col_colname].notna().sum()
    total_cols = all_columns
    print(f"Total de colonnes : {total_cols}")
    total_preds = df[annotation].notna().sum()
    print(f"Total de prédictions : {total_preds}")

    for i in range(1, len(levels)+1):
        allowed = set(levels[:i])
        label = "+".join(levels[:i])

        # Précision : proportion de bonnes prédictions parmi toutes
        true_positive_preds = df[df[col_remarque].isin(allowed)]
        precision = len(true_positive_preds) / total_preds if total_preds else 0

        # Rappel : nombre de colonnes avec au moins une bonne prédiction
        true_positive_cols = true_positive_preds[col_colname].notna().sum()
        recall = true_positive_cols / total_cols if total_cols else 0

        # F1-score
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0

        # Bruit : nb moyen de mauvaises annotations par colonne bien annotée
        bad_preds = df[~df[col_remarque].isin(allowed)]
        bad_per_tp_col = len(bad_preds) / true_positive_cols if true_positive_cols else 0

        results.append({
            "Granularité": label,
            "Précision": round(precision, 3),
            "Rappel": round(recall, 3),
            "F1-score": round(f1, 3),
            "Colonnes bien annotées": true_positive_cols,
            "Annotations incorrectes / col ok": round(bad_per_tp_col, 2)
        })

    return pd.DataFrame(results)





In [ ]:
compute_metrics(df, col_colname="nom colonne",annotation='annotation hybride', col_remarque="Remarque")       # Pour l'hybride


In [ ]:
compute_metrics(df, col_colname="nom colonne",annotation='Annotation Doduo', col_remarque="Remarque.1")     # Pour Doduo


In [ ]:
import pandas as pd

def recall_by_annotation_quality(df, col_colname, col_remarque, seuils=[0.0, 0.25, 0.5, 0.75, 1.0], bon_labels = {"match exact", "spécifique", "générique"}):
    """
    Calcule, pour différents seuils de qualité, le rappel (colonnes bien annotées / colonnes totales).
    
    Une colonne est "bien annotée" si :
    (nombre d'annotations correctes / total annotations) >= seuil
    """
    results = []
    
    total_cols = df[col_colname].notna().sum()
    print(f"Nombre total de colonnes évaluées : {total_cols}")

    # Regrouper les annotations par colonne
    grouped = df.groupby(col_colname)

    for seuil in seuils:
        good_cols = 0
        
        for col_colname, group in grouped:
            total_annots = len(group)
            good_annots = (group[col_remarque].isin(bon_labels)).sum()
            
            proportion = good_annots / total_annots if total_annots else 0
            
            if proportion >= seuil:
                good_cols += 1

        recall = good_cols / total_cols if total_cols else 0

        results.append({
            "Seuil annotations correctes (%)": int(seuil * 100),
            "Colonnes bien annotées": good_cols,
            "Rappel (colonne)": round(recall, 3)
        })

    return pd.DataFrame(results)
